In [1]:
from dfs_service import DFS, distance_obj,filter_dups,find_best_connection_distance,MySolutionMap
from dfs_service import connection_exists
from itertools import permutations
import settings
import time
from decimal import Decimal
from utils import load_maps

In [2]:
distances = distance_obj()
ride_map = load_maps()

In [3]:
 # nested loop,region to region connection
all_regions_connections = permutations([region_name for region_list, region_name in settings.REGIONS_LIST])
all_regions_connections = list(all_regions_connections)
print(f" permutations of all regions {len(list(all_regions_connections))}\n")

 permutations of all regions 362880



In [4]:
# ride_map

In [5]:
solution_map = MySolutionMap()



t0 = time.time()
for region_list, region_name in settings.REGIONS_LIST:
    t1 = time.time()
    d = DFS(region_list)
    solutions = d.start()
    filtered_solutions = filter_dups(solutions, distances)
    # solution_map[region_name] = d.start()
    solution_map[region_name] = filtered_solutions

    print(
        f" {region_name} region took {time.time() - t1}, total solutions {len(solutions)} after filter {len(filtered_solutions)}\n")

print(f" full searches took {time.time() - t0}")

 STAR_WARS_GALAXY_EDGE region took 0.002933979034423828, total solutions 2 after filter 2

 TOON_TOWN region took 0.034837961196899414, total solutions 1728 after filter 54

 CRITTER_COUNTRY region took 0.002032756805419922, total solutions 6 after filter 6

 NEW_ORLEANS_SQUARE region took 0.00191497802734375, total solutions 4 after filter 4

 FRONTIER_LAND region took 0.002048015594482422, total solutions 4 after filter 4

 ADVENTURE_LAND region took 0.0018639564514160156, total solutions 4 after filter 4

 MAIN_STREET region took 0.01076197624206543, total solutions 648 after filter 42

 TOMORROW_LAND region took 0.00600886344909668, total solutions 72 after filter 24

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
 FANTASY_LAND region took 28.97538995742798, total solutions 237973 after filter 24

 full searches took 29.03848910331726


In [6]:
solution_map["NEW_ORLEANS_SQUARE"]

{(69026, 19797): [69026, 6984, 76976, 72179, 19797],
 (69026, 72179): [69026, 6984, 76976, 19797, 72179],
 (72179, 69026): [72179, 19797, 76976, 6984, 69026],
 (19797, 69026): [19797, 72179, 76976, 6984, 69026]}

In [7]:
# permutation of all region connects
# we need to start in main street, so no reason to generate permutations where main street is not first
perms = permutations([name for _, name in settings.REGIONS_LIST if name not in { settings.MAIN_STREET_NAME, settings.TOON_TOWN_NAME}])
perms = list(perms)
print(len(perms))
print(f"all region names {[name for _, name in settings.REGIONS_LIST if name not in { settings.MAIN_STREET_NAME, settings.TOON_TOWN_NAME}]}")
perms = [[settings.MAIN_STREET_NAME] + list(p) for p in perms]
print(len(perms))

print(f"found {len(perms)} region to region paths")
print(f"path contains {len(perms[0]) } elements")

5040
all region names ['STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'FRONTIER_LAND', 'ADVENTURE_LAND', 'TOMORROW_LAND', 'FANTASY_LAND']
5040
found 5040 region to region paths
path contains 8 elements


In [8]:

# print(settings.REGIONS_LIST)

magic = ['MAIN_STREET', 'ADVENTURE_LAND', "FRONTIER_LAND", "NEW_ORLEANS_SQUARE", "CRITTER_COUNTRY", "STAR_WARS_GALAXY_EDGE", "FANTASY_LAND"]
for idx, p in enumerate(perms):
    
    
    if (magic[0] == p[0]) and (magic[1] == p[1]) and (magic[2] == p[2]) and (magic[3] == p[3]):
        if (magic[4] == p[4]) and (magic[5] == p[5]) and (magic[6] == p[6]):
            print(p, idx)
        
        
        
    

['MAIN_STREET', 'ADVENTURE_LAND', 'FRONTIER_LAND', 'NEW_ORLEANS_SQUARE', 'CRITTER_COUNTRY', 'STAR_WARS_GALAXY_EDGE', 'FANTASY_LAND', 'TOMORROW_LAND'] 3295


In [9]:
# starting in main street,
# inside each solution for source region,check exit node
# check if exit node has connection with entry node from next region
# if none found, exit path

best_distance = float("inf")
my_inf = Decimal(float("inf"))
deadend = set()
longest_path = float("-inf")
# print(len(perms))
# print(perms[0])
best_solutions = []

FLOW_SOLUTION = []


for pidx, path in enumerate(perms):
    
    CURR_FLOW = []
    
    curr_path_distance = Decimal("0.0")
#     tmp_distance = 0
    for i in range(1, len(path)):
        r_source = path[i - 1]
        r_target = path[i]
        
        if (r_source, r_target) in deadend:
            curr_path_distance = my_inf
            if pidx == 3295:
                print("FALSE deadend")
            break

        tmp_distance = find_best_connection_distance(r_source, r_target, solution_map)
        curr_path_distance += tmp_distance
        
        if tmp_distance == my_inf:
            print(f"no connection between {r_source} {r_target}, {i}th in path {pidx}")
            deadend.add((r_source, r_target))
            if i >= 7:
                print(f"{path}")
            
            print("\n")
            longest_path = max(longest_path, i)
#             curr_path_distance = float("inf")
            break
            
#         else:

        
    
    if  curr_path_distance != my_inf  and (i == len(path)-1):
#     if  curr_path_distance != my_inf:
        best_distance = min(best_distance, curr_path_distance)
        if curr_path_distance == best_distance:
            best_path = path
            
        best_solutions.append( list(path) )
        
        
#     if pidx == 3295:
#         print( "@@@", i , len(path)-1  ,path, curr_path_distance )

print(f"best distance path: {best_distance}")
print(f"longest reached: {longest_path}")
print(f"scanned: {pidx}")

no connection between MAIN_STREET STAR_WARS_GALAXY_EDGE, 1th in path 0


no connection between MAIN_STREET CRITTER_COUNTRY, 1th in path 720


no connection between MAIN_STREET NEW_ORLEANS_SQUARE, 1th in path 1440


no connection between NEW_ORLEANS_SQUARE TOMORROW_LAND, 5th in path 2162


no connection between NEW_ORLEANS_SQUARE FANTASY_LAND, 5th in path 2164


no connection between CRITTER_COUNTRY ADVENTURE_LAND, 4th in path 2166


no connection between CRITTER_COUNTRY TOMORROW_LAND, 4th in path 2172


no connection between CRITTER_COUNTRY FANTASY_LAND, 4th in path 2178


no connection between STAR_WARS_GALAXY_EDGE NEW_ORLEANS_SQUARE, 3th in path 2184


no connection between STAR_WARS_GALAXY_EDGE ADVENTURE_LAND, 3th in path 2208


no connection between STAR_WARS_GALAXY_EDGE TOMORROW_LAND, 3th in path 2232


no connection between FANTASY_LAND CRITTER_COUNTRY, 4th in path 2256


no connection between FANTASY_LAND NEW_ORLEANS_SQUARE, 4th in path 2262


no connection between ADVENTURE_LAN

In [10]:
### use DFS to find all combinations

all_possibilities = []

#path:   main street, tomorrowland, fantasyland ....
for pidx, path in enumerate(perms):
    
        path_possibilities = [  solution_map[path[0] ]  ]
        
        for i in range(1, len(path)):
            r_source = path[i - 1]
            r_target = path[i]
            #source, target = main street, tomorrowland
            
            #is it a dead end?
            ###
            if not connection_exists(r_source, r_target, solution_map):
                continue
            
            
            
            
            #get connections:
            #source = [  [entrance, store1 ], [entrance, store2]   ]
            #target = [  [orbital, space mountain], [ buzz, nemo]   ]
            
            path_possibilities.append( solution_map[r_target] )
        
        
        if len(path) == len(path_possibilities):
            all_possibilities.append( path_possibilities  )
        
            







In [16]:
print(len(all_possibilities))

45


In [11]:
print(deadend)

{('NEW_ORLEANS_SQUARE', 'FANTASY_LAND'), ('FANTASY_LAND', 'NEW_ORLEANS_SQUARE'), ('STAR_WARS_GALAXY_EDGE', 'TOMORROW_LAND'), ('TOMORROW_LAND', 'CRITTER_COUNTRY'), ('NEW_ORLEANS_SQUARE', 'TOMORROW_LAND'), ('CRITTER_COUNTRY', 'TOMORROW_LAND'), ('TOMORROW_LAND', 'NEW_ORLEANS_SQUARE'), ('MAIN_STREET', 'NEW_ORLEANS_SQUARE'), ('ADVENTURE_LAND', 'STAR_WARS_GALAXY_EDGE'), ('STAR_WARS_GALAXY_EDGE', 'NEW_ORLEANS_SQUARE'), ('TOMORROW_LAND', 'FRONTIER_LAND'), ('FRONTIER_LAND', 'TOMORROW_LAND'), ('FANTASY_LAND', 'CRITTER_COUNTRY'), ('FANTASY_LAND', 'FRONTIER_LAND'), ('STAR_WARS_GALAXY_EDGE', 'ADVENTURE_LAND'), ('TOMORROW_LAND', 'STAR_WARS_GALAXY_EDGE'), ('FRONTIER_LAND', 'FANTASY_LAND'), ('MAIN_STREET', 'CRITTER_COUNTRY'), ('MAIN_STREET', 'STAR_WARS_GALAXY_EDGE'), ('CRITTER_COUNTRY', 'ADVENTURE_LAND'), ('CRITTER_COUNTRY', 'FANTASY_LAND'), ('NEW_ORLEANS_SQUARE', 'STAR_WARS_GALAXY_EDGE'), ('ADVENTURE_LAND', 'CRITTER_COUNTRY')}


In [12]:
print(len(best_solutions))
print(best_path)

45
['MAIN_STREET', 'TOMORROW_LAND', 'FANTASY_LAND', 'ADVENTURE_LAND', 'FRONTIER_LAND', 'NEW_ORLEANS_SQUARE', 'CRITTER_COUNTRY', 'STAR_WARS_GALAXY_EDGE']


In [13]:

conns = (
#     ("CRITTER_COUNTRY", "NEW_ORLEANS_SQUARE"),
#     ("ADVENTURE_LAND", "NEW_ORLEANS_SQUARE"),
#     ("ADVENTURE_LAND", "FRONTIER_LAND"),
#      ("NEW_ORLEANS_SQUARE", "FRONTIER_LAND"),
#      ("NEW_ORLEANS_SQUARE", "ADVENTURE_LAND"),
#          ("NEW_ORLEANS_SQUARE", "CRITTER_COUNTRY"),
#              ("MAIN_STREET", "ADVENTURE_LAND"),
#     ("ADVENTURE_LAND", "FRONTIER_LAND"),
#     ("FRONTIER_LAND", "NEW_ORLEANS_SQUARE"),
#     ("NEW_ORLEANS_SQUARE","CRITTER_COUNTRY",),
#     ("CRITTER_COUNTRY", "STAR_WARS_GALAXY_EDGE"),
#     ( "STAR_WARS_GALAXY_EDGE", "FANTASY_LAND"),
#     ("FANTASY_LAND", "TOMORROW_LAND"),
    
#         ("NEW_ORLEANS_SQUARE", "STAR_WARS_GALAXY_EDGE"),
#                  ("MAIN_STREET", "NEW_ORLEANS_SQUARE"),
    
    ("MAIN_STREET", "TOMORROW_LAND"),
    ("FANTASY_LAND", "TOMORROW_LAND"),
    ("FANTASY_LAND", "ADVENTURE_LAND"),
    ("FRONTIER_LAND", "ADVENTURE_LAND"),
    ("FRONTIER_LAND", "CRITTER_COUNTRY"),
    ("NEW_ORLEANS_SQUARE", "CRITTER_COUNTRY"),
    ("NEW_ORLEANS_SQUARE", "STAR_WARS_GALAXY_EDGE"),
    ("FANTASY_LAND", "TOON_TOWN"),


    
    
    


)

for r_source, r_target in conns:
    tmp_distance = find_best_connection_distance(r_source, r_target, solution_map)
    print(r_source, r_target,  tmp_distance, (r_source, r_target) in deadend )

MAIN_STREET TOMORROW_LAND 0.11178482480398058 False
FANTASY_LAND TOMORROW_LAND 0.08661118592832563 False
FANTASY_LAND ADVENTURE_LAND 0.11867273923671834 False
FRONTIER_LAND ADVENTURE_LAND 0.05174383191634434 False
FRONTIER_LAND CRITTER_COUNTRY 0.09229982679104619 False
NEW_ORLEANS_SQUARE CRITTER_COUNTRY 0.05021377438156316 False
NEW_ORLEANS_SQUARE STAR_WARS_GALAXY_EDGE Infinity True
FANTASY_LAND TOON_TOWN 0.07229029191445366 False


In [14]:
for key in solution_map['FANTASY_LAND']:
    
    k1, k2 = key
    k1,k2 = ride_map[k1]['name'],ride_map[k2]['name']
    
    print(f"{(k1,k2)}: {[   ride_map[e]['name']  for e in solution_map['FANTASY_LAND'][key]    ]}\n\n")


('sleeping beauty castle walkthrough', "it's a small world"): ['sleeping beauty castle walkthrough', "mr. toad's wild ride", 'mad tea party', 'matterhorn bobsleds', 'alice in wonderland', 'dumbo the flying elephant', "snow white's scary adventures", "peter pan's flight", "casey's jr train", "pinocchio's daring journey", 'king arthur carrousel', 'storybook land canal boats', 'its a small world holiday', "it's a small world"]


('sleeping beauty castle walkthrough', 'its a small world holiday'): ['sleeping beauty castle walkthrough', "mr. toad's wild ride", 'mad tea party', 'matterhorn bobsleds', 'alice in wonderland', 'dumbo the flying elephant', "snow white's scary adventures", "peter pan's flight", "casey's jr train", "pinocchio's daring journey", 'king arthur carrousel', 'storybook land canal boats', "it's a small world", 'its a small world holiday']


('mad tea party', "it's a small world"): ['mad tea party', 'matterhorn bobsleds', 'alice in wonderland', 'dumbo the flying elephant',

In [15]:
for key in solution_map['ADVENTURE_LAND']:
    
    k1, k2 = key
    k1,k2 = ride_map[k1]['name'],ride_map[k2]['name']
    
    print(f"{(k1,k2)}: {[   ride_map[e]['name']  for e in solution_map['ADVENTURE_LAND'][key]    ]}\n\n")

("tarzan's tree house", "walt disney's enchanted tiki room"): ["tarzan's tree house", 'indiana jones™ adventure', 'jungle cruise', "walt disney's enchanted tiki room"]


("walt disney's enchanted tiki room", "tarzan's tree house"): ["walt disney's enchanted tiki room", 'jungle cruise', 'indiana jones™ adventure', "tarzan's tree house"]


("walt disney's enchanted tiki room", 'indiana jones™ adventure'): ["walt disney's enchanted tiki room", 'jungle cruise', "tarzan's tree house", 'indiana jones™ adventure']


('indiana jones™ adventure', "walt disney's enchanted tiki room"): ['indiana jones™ adventure', "tarzan's tree house", 'jungle cruise', "walt disney's enchanted tiki room"]


